In [1]:
import awkward as ak
import dask
from distributed import performance_report
import hist
from hist.dask import Hist
import json
from coffea import processor
from coffea.nanoevents import BaseSchema, NanoAODSchema 
from coffea.dataset_tools import apply_to_dataset, apply_to_fileset, preprocess
from coffea.dataset_tools import max_chunks, max_files, slice_chunks, slice_files
from coffea.analysis_tools import Weights, PackedSelection
import matplotlib.pyplot as plt

/usr/local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
from msdprocessor import msdProcessor

/usr/local/lib/python3.10/site-packages/coffea/nanoevents/methods/candidate.py:11: FutureWarning: In version 2024.7.0 (target date: 2024-06-30 11:59:59-05:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


In [3]:
from dask.distributed import Client

client = Client("tls://localhost:8786")
client

Connection method: Direct,
Dashboard: /user/ruvarashe.nyaban@bulldogs.aamu.edu/proxy/8787/status,
Comm: tls://192.168.121.100:8786,Workers: 0
Dashboard: /user/ruvarashe.nyaban@bulldogs.aamu.edu/proxy/8787/status,Total threads: 0
Started: 32 minutes ago,Total memory: 0 B


In [10]:
with open("2018_GluGluHToBB.json", "rt") as file:
    initial_fileset = json.load(file)

In [11]:
# def validate_files(initial_fileset):
#     validated_files = {}

#     for dataset, paths in initial_fileset.items():
#         validated_files[dataset] = {"files": {}}
#         for file_path in paths:
#             if is_valid_file(file_path):
#                 validated_files[dataset]["files"][file_path] = "Events"
#             else:
#                 print(f"Invalid file path skipped: {file_path}")
    
#     return validated_files

In [12]:
# print(type(initial_fileset))

In [17]:
 # print(json.dumps(initial_fileset, indent=2))

In [14]:
files={}
for i, j in initial_fileset.items():
    print(i)
    files[i]={"files":{}}
    for k in j:
        print(k)
        files[i]["files"][k] = "Events"

GluGluHToBB
root://cmsxrootd.fnal.gov//store/group/lpcpfnano/jdickins/v2_2/2018/GluGluHToBB/GluGluHToBB_Pt-200ToInf_M-125_TuneCP5_MINLO_13TeV-powheg-pythia8/GluGluHToBB/220322_131235/0000/nano_mc2018_1-1.root
root://cmsxrootd.fnal.gov//store/group/lpcpfnano/jdickins/v2_2/2018/GluGluHToBB/GluGluHToBB_Pt-200ToInf_M-125_TuneCP5_MINLO_13TeV-powheg-pythia8/GluGluHToBB/220322_131235/0000/nano_mc2018_1.root
root://cmsxrootd.fnal.gov//store/group/lpcpfnano/jdickins/v2_2/2018/GluGluHToBB/GluGluHToBB_Pt-200ToInf_M-125_TuneCP5_MINLO_13TeV-powheg-pythia8/GluGluHToBB/220322_131235/0000/nano_mc2018_10.root
root://cmsxrootd.fnal.gov//store/group/lpcpfnano/jdickins/v2_2/2018/GluGluHToBB/GluGluHToBB_Pt-200ToInf_M-125_TuneCP5_MINLO_13TeV-powheg-pythia8/GluGluHToBB/220322_131235/0000/nano_mc2018_11.root
root://cmsxrootd.fnal.gov//store/group/lpcpfnano/jdickins/v2_2/2018/GluGluHToBB/GluGluHToBB_Pt-200ToInf_M-125_TuneCP5_MINLO_13TeV-powheg-pythia8/GluGluHToBB/220322_131235/0000/nano_mc2018_12.root
root://c

In [15]:
# print(files)

In [20]:
preprocessed_available, preprocessed_total = preprocess(
        files,
        step_size=50_000,
        align_clusters=None,
        skip_bad_files=True,
        recalculate_steps=False,
        files_per_batch=1,
        file_exceptions=(OSError,),
        save_form=True,
        uproot_options={},
        step_size_safety_factor=0.5,
    )

IndexError: cannot slice EmptyArray (of length 0) with ['file', 'object_path', 'steps', 'num_entries', 'uuid']: not an array of records

This error occurred while attempting to slice

    <Array [None, None, None, None, ..., None, None, None] type='22 * ?unknown'>

with

    ['file', 'object_path', 'steps', 'num_entries', 'uuid']

In [ ]:
import gzip, pickle, json
output_file = "scaleout_fileset"
with gzip.open(f"{output_file}_available.json.gz", "wt") as file:
    json.dump(preprocessed_available, file, indent=2)
    print(f"Saved available fileset chunks to {output_file}_available.json.gz")
with gzip.open(f"{output_file}_all.json.gz", "wt") as file:
    json.dump(preprocessed_total, file, indent=2)
    print(f"Saved complete fileset chunks to {output_file}_all.json.gz")

In [ ]:
test_preprocessed_files = max_files(preprocessed_available, 30)
test_preprocessed = max_chunks(test_preprocessed_files, 30)

In [ ]:
tg, rep = apply_to_fileset(data_manipulation=ggF(),    
                            fileset=test_preprocessed,
                            schemaclass=NanoAODSchema,
                            uproot_options={"allow_read_errors_with_report": (OSError, KeyError)},
                           )

## Distributed performance_report
The dask distributed performance_report will save an html page of our dask computations,
preserving a record of the tasks, performance profiling, and so on

In [ ]:
performance_report_name = "ggf_report.html"
with performance_report(performance_report_name, stacklevel=1, mode=None, storage_options=None):
    result, report = dask.compute(tg, rep)

### Computing the luminosity scaling
The coffea lumi_tools subpackage contains tools for calculating the lumi processed
We'll take a simpler approach, calculating the fraction of the dataset we computed
and scaling the known-luminosity of the 2018A run by this fraction

In [ ]:
# Compute h
def total_data(events):
    isRealData = "genWeight" not in events.fields
    if isRealData:
        events = events[
            corrections.lumimask(events.run, events.luminosityBlock)
        ]
        return ak.num(events, axis=0)
    else:
        return -1

In [ ]:
# This number should be calculated on the preprocessed_total,
# but some of those files may be inaccessible. We rely upon most being readable
dfd, _ = apply_to_fileset(data_manipulation=total_data,
                                     fileset=preprocessed_available,
                                     schemaclass=NanoAODSchema,
                                     uproot_options={"allow_read_errors_with_report": (OSError, KeyError)},
                                    )
data_fraction_den = dfd["DoubleMuon2018A"].compute()

In [ ]:

data_fraction_num = result["DoubleMuon2018A"]["cutflow"]["lumimask"]
data_fraction = data_fraction_num / data_fraction_den
print(data_fraction)

In [ ]:
data = result["DoubleMuon2018A"]["mass"]

lumi = 14.0
xsweight = lumi * 1e3 * 6225.42 * data_fraction / result["ZJets2018"]["sumw"]["genWeight"]["sumw"]
sim = result["ZJets2018"]["mass"] * xsweight

In [ ]:
fig, ax = plt.subplots()
sim.plot(ax=ax, histtype="fill", label="Z+jets")
data.plot(ax=ax, histtype="errorbar", color="k", label="Data")
ax.set_xlim(60, 120)
ax.legend()